<a href="https://colab.research.google.com/github/Benned-H/Summer2019/blob/master/Speech%20and%20Language%20Processing/Chapter_3_Words_%26_Transducers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Words & Transducers (p. 51)

We've seen how to create plural *woodchucks*, but plural *foxes*, *peccaries*, and *geese* are all irregular forms. *Fish* even retains its spelling in the plural! To search for these forms, we need **orthographic rules** (*-y* endings turn to *-ie* in the plural) and **morphological rules** (*fish* has a null plural, *geese* has a changed vowel).

**Morphological parsing**, or stemming, is the problem of recognizing and separating morphemes of words in order to build structured representations. **Parsing** itself entails taking an input and producing some linguistic structure for it. In this case, we could take the **surface form** *going* and parse it into ```VERB-go + GERUND-ing```.

This technique is especially important for morphologically complex languages, as we want to search for a single word and find its many inflections as well. It's also used in POS-tagging and machine translation. To solve morphological parsing, couldn't we use a dictionary of valid forms? The issue is that many affixes are **productive**, e.g. we can take a word *Google* and create *Googled* or *fax* and create *faxing*. Both *-ing* and *-s* can attach to almost any verb or noun, respectively. Other languages make the dictionary idea even more infeasible.

*Overview*: In this chapter we'll survey the morphology of English and some other languages and then introduce **finite-state transducers**, a crucial technology in many areas of this text. We'll also introduce related algorithms like **stemming**, which maps from derived words to their stem. Similarly, the problem of mapping *sing*, *sang*, and *sung* to the same **lemma** is called **lemmatization**. Next, **tokenization** is the task of separating words from a text. Whitespace isn't infallible, as words like *New York* or *rock 'n' roll* contain spaces, yet two-word contractions often *don't*. Finally, we sometimes want to see how similar two words are orthographically (spelling-wise), and the **minimum edit distance** algorithm works for such spell-checking applications.

## 3.1 Survey of (Mostly) English Morphology (p. 53)

Morphology studies how words are built from smaller units of meaning called **morphemes**. These "minimal meaning-breaing units" fall into two broad classes, **stems** (or roots, bases) and **affixes**. Intuitively, stems are the main morphemes of words while affixes add additional kinds of meaning. We can further divide affixes into **prefixes**, **suffixes**, **infixes**, and **circumfixes**. Prefixes precede their stem, suffixes follow, circumfixes do both, and infixes are inserted into the stem. The first two might be obvious in English, but we need to look to German for a circumfix: the past participle of some verbs is formed by adding *ge*- to the beginning and -*t* to the end of the base, eg. *sagen* (to say) becomes *gesagt* (said).

We can stack a few affixes in English, maybe *un-* *believe* *-able* *-ly* creating *unbelievably*, but other languages like Turkish can have nine or ten affixes. Such many-affix langauges are called **agglutinative**. We can also find define a few ways, such as **inflection**, **derivation**, **compounding**, and **cliticization**, to create new words by combining morphemes. **Inflection** combines a stem with a grammatical morpheme, usually in the same word class as the original word. **Derivation** combines a stem with a grammatical morpheme resulting in a different class of word, like *create* -*tion* becoming *creation*. **Compounding** combines multiple stems, and **cliticization** combine a word stem with a **clitic**. Clitics are morphemes that act like words but are reduced in form, like *'ve'* in *I've* or the French definite article *l'* in *l'opera*.

### Inflectional Morphology

English has a limited inflectional system. English nouns have two kinds of inflection: **plural** and **possessive**. Many English nouns can either appear in a bare **singular** form or take a plural suffix. The plural suffix often takes the form -*s*, but can be -*es* or change -*y* to -*ies* depending on the end of words. The possessive is realized by an apostrophe + -*s* in many cases.


Verbal inflection brings a few more options to English, as English has three kinds of verbs: **main verbs** (*eat*, *sleep*, *impeach*), **modal verbs** (*can*, *will*, *should*), and **primary verbs** (*be*, *have*, *do*) (terms of Quirk et al., 1985). Most verbs are **regular**, meaning they have the same endings marking the same functions. These four forms are the stem (*walk*, *try*), -*s* form (*walks*, *tries*), -*ing* participle (*walking*, *trying*), and past form (*walked*, *tried*). This class of verbs is **productive**, meaning it automatically includes new words introduced to the language.

**Irregular verbs** have different, abnormal forms of inflection. These can have as many as eight forms, few as three, but are often some of the most often-used words. As an example, *eat* has the past *ate* as well as the -*ed*/-*en* participle *eaten*. The usage of all of these forms is as follows:
* -*s* is used to distinguish 3rd-person singular.
* Stem forms are used as infinitives.
* -*ing* is used in the **progressive** to mark ongoing activity, or when the verb is used as a noun as a **gerund** (*Fishing* is fun).
* -*ed*/-*en* is used in the **perfect** or passive (He's *eaten* already; The apple was *eaten*).

We also need to consider spelling differences as we attach these different suffixes. Single consonants are doubled, a final 'c' becomes 'ck', delete a silent final -*e*, etc.

### Derivational Morphology

To be continued...

*--Last updated 6/13/2019--*